In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read

[anooja-ThinkPad-X1-Carbon-Gen-11:16236] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)


In [12]:
atoms = read("../dft/device/scatt.xyz")
y_threshold = 4.0

# Get atomic symbols and z coordinates
symbols = atoms.get_chemical_symbols()
x_coords = atoms.positions[:, 0]
y_coords = atoms.positions[:, 1]

# Count atoms in the leads
num_atoms_bridge = np.sum(y_coords > y_threshold)

# Atoms per lead (assuming symmetric leads)
num_atoms_left_lead = 5
num_atoms_right_lead = 4


num_spherical_harmonics = {"Au": 6, "C": 4, "O": 4}

# Count spherical harmonics in left lead
left_lead_indices = np.argsort(x_coords)[:num_atoms_left_lead]
total_sph_left_lead = sum(num_spherical_harmonics[symbols[i]] for i in left_lead_indices)

# Count spherical harmonics in the bridge
bridge_indices = np.argsort(x_coords)[num_atoms_left_lead:num_atoms_left_lead + num_atoms_bridge]
total_sph_bridge = sum(num_spherical_harmonics[symbols[i]] for i in bridge_indices)

# Count spherical harmonics in right lead
right_lead_indices = np.argsort(x_coords)[-num_atoms_right_lead:]
total_sph_right_lead = sum(num_spherical_harmonics[symbols[i]] for i in right_lead_indices)

# Populate nodes
nodes = [
    0,
    total_sph_left_lead,
    total_sph_left_lead + total_sph_bridge,
    total_sph_left_lead + total_sph_bridge + total_sph_right_lead]
nodes

[0, 30, 38, 62]

In [13]:
num_atoms_bridge

2

In [14]:
left_lead_indices, bridge_indices, right_lead_indices

(array([0, 1, 2, 3, 4]), array([5, 6]), array([ 7,  8,  9, 10]))

In [ ]:
output_folder = "../block_tridiagonal_gf/output/no_lowdin"
np.save(f"{output_folder}/nodes.npy", nodes)